<a href="https://colab.research.google.com/github/kabilan942/Legal/blob/main/Indian%20Legal%20Abs%20Ext/LDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## utilities.py

Defined Functions in utilities.py:

`get_root_path()`

`get_summary_data(dataset, train)`

`get_summary_data_rhet_train(dataset)
`
`get_req_len_dict(dataset, istrain)`

`split_to_sentences(para)`

`nest_sentencesV2(document,chunk_length)`

`nest_sentencesMV2(document_sents,chunk_length)`

`nest_sentencesV3(doc_sents,chunk_length, dict_sents_labels)`

`get_doc_sens_and_labels(doc)`

In [1]:
import glob
from nltk import tokenize
import nltk
!pip install transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch

def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset"
    return path

def get_summary_data(dataset, train):
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''
    if dataset == "N2":
        path = get_root_path() + '/N2/Full-Text/India'
        all_files = glob.glob(path + "/*.txt")

        data_source = []
        names = []
        for filename in all_files:
            with open(filename, 'r') as f: 
                p = filename.rfind("/")
#                 print(filename[p+1:])
                names.append(filename[p+1:])
                a = f.read()
                data_source.append(a)
        return names, data_source, []
    
    path = get_root_path() + '/Summary-Data-' + dataset + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/Summary-Data-' + dataset + '/' + train + '-data/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

def get_summary_data_rhet_train(dataset):
    '''
    function to get names, documents, and summaries for Rhetorical labeled documents - train
    change the path variable to the path of the Rhetorical labeled dataset
    '''
    path = get_root_path() + '/rhet/' + dataset.lower() + '_ft_rhet' # use your path
    all_files = glob.glob(path + "/*.txt")

    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)

    path = get_root_path() + '/rhet/RhetSumm_Dataset/raw_files/'+ dataset +'/summary' # use your path
    all_files = glob.glob(path + "/*.txt")

    data_summary = {}
    for filename in all_files:
        with open(filename, 'r') as f: 
            p = filename.rfind("/")
            a = f.read()
            l = len(a)
            data_summary[filename[p+1:]] = (a)
    return names, data_source, data_summary

def get_summary_data_rhet_test(dataset):
    '''
    function to get names, documents, and summaries for Rhetorical labeled documents - test
    change the path variable to the path of the Rhetorical labeled dataset
    '''
    path = get_root_path() + '/rhet/RhetSumm_Dataset/rhet/' + dataset + "/" # use your path
    all_files = glob.glob(path + "/*.txt")

    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            p = filename.rfind("/")
#             print(filename[p+1:])
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)

    return names, data_source


def get_req_len_dict(dataset, istrain):
    '''
    function to required length data for each summary
    change the path variable to the path to Summary_Length_India.txt/ Summary_Length_Uk.txt file
    '''
    
    if dataset == "N2":
        f = open(get_root_path() + "/N2/Summary_Length_India.txt", "r")
        a = (f.read())
        a = a.split("\n")
        dict_names = {}
        for i in a:
            b = i.split("	")
            dict_names[b[0] + ".txt"] = int(b[1])
        return dict_names 
    
    f = open(get_root_path() + "/Summary-Data-"+ dataset +"/length-file-" + istrain +".txt", "r")
    a = (f.read())
    a = a.split("\n")
    dict_names = {}
    for i in a:
        b = i.split("	")
        try:
            tp = int(b[2])
            dict_names[b[0]] = tp
        except:
            print(b)
    return dict_names  

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)
    return sents

def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

def nest_sentencesMV2(document_sents,chunk_length):
    '''
    function to chunk a document
    input:  doc_sents           - Input document sentences
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    #modeified v2
    
    for sentence in document_sents:
        length += len((sentence.split(" ")))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

def nest_sentences(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a string.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested
  

def nest_sentencesV3(doc_sents,chunk_length, dict_sents_labels):
    '''
    function to first segment the document using rhetorical roles and then chunk if required
    input:  doc_sents           - Input document sentences
            chunk_length        - chunk length
            dict_sents_labels   - dictionary for every sentence and label
    output: list of chunks
    '''
    s = list(set(dict_sents_labels.values()))
#     print(s)
    all_chunks = []
    
    for label in s:
        doc_sents_withlabels = []
        for sent in doc_sents:
            if sent == '':continue
            if dict_sents_labels[sent] == label:
                doc_sents_withlabels.append(sent)
        chunks = nest_sentencesMV2(doc_sents_withlabels, chunk_length)
        
        edited_chunks = []
        for chunk in chunks:
            edited_chunks.append(["<" + label + ">"] + chunk)
        
        all_chunks = all_chunks + edited_chunks

    return all_chunks   

def get_doc_sens_and_labels(doc):
    '''
    Function to read a Rhetorically labeled document.
    returns a list of sentences, the label of each sentence, dictionary: keys-sentences, values-labels
    '''
    sents = []
    labels = []
    dict_sents_labels = {}
    ss = doc.split("\n")
    for i in ss:
        try:
            spt = i.split("\t")
            sents.append(spt[0])
            labels.append(spt[1])
            dict_sents_labels[spt[0]] = spt[1] 
        except:
            pass
    return sents, labels, dict_sents_labels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.4 MB/s eta 0:00:00


## Chunker.ipynb
Script to chunk and save every file. Every line in the output file is a chunk.


In [2]:
import pandas as pd
import numpy as np
import glob
import nltk
import os
import sys
sys.path.insert(0, '../')
# from utilities import *

In [34]:
def get_root_path():
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs"
    return path

def get_summary_data(dataset, train):  
    path = get_root_path() + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

import nltk
nltk.download('punkt')

def nest_sentences(document,chunk_length):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
dataset = "IN" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext 
# output_path = "./output/"
output_path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/test-data"

names, data_source, data_summary = get_summary_data(dataset, "test")    # names, documents and summaries

print(len(names))
print(len(data_source))
print(len(data_summary))


100
100
100


In [39]:
import os
if not os.path.exists(output_path + '/'+ dataset +'_chunked/'):
    os.makedirs(output_path + '/'+ dataset +'_chunked/')

for i in range(len(data_source)):
    doc = data_source[i].replace("\n", " ")
    chunks = nest_sentences(doc, 512)
    save = "\n".join(chunks)
    path = output_path +  "/"+ dataset +"_chunked/" + names[i]
    file = open(path,'w')
    file.write(save)
    file.close()

## Pointer Generator

### Chunker

Each chunk would have its own text file

In [21]:
# output_path = "./chunked/"
output_path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [32]:
def get_root_path():
    # path = "/home/pahelibhattacharya/Abhayv2/camera_ready/summarization/dataset"
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs"
    return path

def get_summary_data(dataset, train):  
    path = get_root_path() + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/' + train + '-data/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

import nltk
nltk.download('punkt')

def nest_sentences(document,chunk_length):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(" ".join(sent))
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(" ".join(sent))
    return nested

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
names, data_source, data_summary = get_summary_data("IN", "test")
print(len(names))
print(len(data_source))
print(len(data_summary))

100
100
100


In [24]:
names[0]

'3210.txt'

In [25]:
data_source[0]

'Appeal No. 1604 of 1967.\nFrom the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.\nP.P. Rao and R. N. Sachthey, for the appellants.\nG.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.\n1 & 2.\nThe Judgment of the Court was delivered by SARKARIA J.\nWhether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.\nThe facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.\nThe primary objects of the Federation as given in the Memorandum of Association are "(a) To promote Indian 

In [26]:
data_summary[0]

'Appeal No. 1604 of 1967.\nFrom the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.\nP.P. Rao and R. N. Sachthey, for the appellants.\nG.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.\n1 & 2.\nThe Judgment of the Court was delivered by SARKARIA J.\nWhether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.\nThe facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.\nThe primary objects of the Federation as given in the Memorandum of Association are "(a) To promote Indian 

In [28]:
output = []
for i in range(len(data_source)):
    name = names[i]
    doc = data_source[i]
    wc = doc.split(" ")
    print(str(i) + ": " + name)
    nested = nest_sentences(doc,395)
    
    for i in range(len(nested)):
        path = output_path + name[:-4] + "_" + str(i) + ".txt"
        print(path)
        file = open(path,'w')
        file.write(nested[i])
        file.close()
    
print(output)

0: 3210.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_0.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_1.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_2.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_3.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_4.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_5.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_6.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_7.txt
/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Pointer_Generator/chunked/3210_8.txt
/content/drive/MyDrive/Le

The 100 documents in test data of IN-Abs has been chunked into several docs with max length of each doc being 395 words.